# Women's Education Data Cleaning

This notebook cleans and prepares two education datasets:
- Women's Advanced Education Labor Force Participation
- Women's Basic Education Labor Force Participation

**Goal:** Create a clean, standardized dataset in long format with both education metrics.

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Load the two education datasets
# Skip the metadata rows (first 4 rows) and use row 5 as header
advanced_ed = pd.read_csv('../data/raw/women_advanced_education.csv', skiprows=4)
basic_ed = pd.read_csv('../data/raw/women_basic_education.csv', skiprows=4)

print("Advanced Education shape:", advanced_ed.shape)
print("Basic Education shape:", basic_ed.shape)
print("\nFirst few columns:")
print(advanced_ed.columns[:8].tolist())

## Step 1: Filter Out Regional Aggregates

Remove regional/income group aggregates and keep only individual countries.

In [2]:
# Define regional/aggregate codes to exclude
# These are common World Bank regional and income group aggregates
regional_codes = [
    'AFE', 'AFW', 'ARB', 'CEB', 'CSS', 'EAP', 'EAR', 'EAS', 'ECA', 'ECS', 
    'EMU', 'EUU', 'FCS', 'HIC', 'HPC', 'IBD', 'IBT', 'IDA', 'IDB', 'IDX',
    'INX', 'LAC', 'LCN', 'LDC', 'LIC', 'LMC', 'LMY', 'LTE', 'MEA', 'MIC',
    'MNA', 'NAC', 'OED', 'OSS', 'PRE', 'PSS', 'PST', 'SAS', 'SSA', 'SSF',
    'SST', 'TEA', 'TEC', 'TLA', 'TMN', 'TSA', 'TSS', 'UMC', 'WLD'
]

def filter_countries(df):
    """Remove regional aggregates and keep only individual countries"""
    # Filter out known regional codes
    df_filtered = df[~df['Country Code'].isin(regional_codes)].copy()
    
    # Also remove any rows where Country Name contains common aggregate keywords
    aggregate_keywords = ['income', 'dividend', 'IBRD', 'IDA', 'Fragile', 'World', 
                         'states', 'Europe &', 'East Asia', 'Latin America', 
                         'Middle East', 'North America', 'South Asia', 'Sub-Saharan']
    
    for keyword in aggregate_keywords:
        df_filtered = df_filtered[~df_filtered['Country Name'].str.contains(keyword, case=False, na=False)]
    
    return df_filtered

# Filter both datasets
advanced_ed_filtered = filter_countries(advanced_ed)
basic_ed_filtered = filter_countries(basic_ed)

print(f"Advanced Education: {len(advanced_ed)} → {len(advanced_ed_filtered)} countries")
print(f"Basic Education: {len(basic_ed)} → {len(basic_ed_filtered)} countries")
print(f"\nSample countries remaining:")
print(advanced_ed_filtered[['Country Name', 'Country Code']].head(10))

Advanced Education: 266 → 215 countries
Basic Education: 266 → 215 countries

Sample countries remaining:
            Country Name Country Code
0                  Aruba          ABW
2            Afghanistan          AFG
4                 Angola          AGO
5                Albania          ALB
6                Andorra          AND
8   United Arab Emirates          ARE
9              Argentina          ARG
10               Armenia          ARM
11        American Samoa          ASM
12   Antigua and Barbuda          ATG


## Step 2: Transform from Wide to Long Format

Convert year columns (1960-2023) into rows to create a tidy dataset.

In [3]:
def reshape_to_long(df, value_name):
    """
    Transform dataset from wide to long format
    
    Parameters:
    - df: DataFrame with years as columns
    - value_name: Name for the value column in long format
    
    Returns:
    - Long format DataFrame with columns: Country Code, Country Name, Year, value_name
    """
    # Keep only essential columns: Country Name, Country Code, and year columns
    # Year columns are all numeric strings from 1960-2024
    year_cols = [col for col in df.columns if col.isdigit()]
    id_cols = ['Country Name', 'Country Code']
    
    # Select relevant columns
    df_subset = df[id_cols + year_cols].copy()
    
    # Melt to long format
    df_long = pd.melt(
        df_subset,
        id_vars=id_cols,
        value_vars=year_cols,
        var_name='Year',
        value_name=value_name
    )
    
    # Convert Year to integer
    df_long['Year'] = df_long['Year'].astype(int)
    
    # Replace empty strings with NaN
    df_long[value_name] = df_long[value_name].replace('', np.nan)
    
    # Convert value column to float
    df_long[value_name] = pd.to_numeric(df_long[value_name], errors='coerce')
    
    return df_long

# Transform both datasets
advanced_long = reshape_to_long(advanced_ed_filtered, 'advanced_education_pct')
basic_long = reshape_to_long(basic_ed_filtered, 'basic_education_pct')

print("Advanced Education Long Format:")
print(f"Shape: {advanced_long.shape}")
print(f"Columns: {advanced_long.columns.tolist()}")
print(f"\nSample data:")
print(advanced_long.head(10))
print(f"\nData types:")
print(advanced_long.dtypes)

Advanced Education Long Format:
Shape: (13975, 4)
Columns: ['Country Name', 'Country Code', 'Year', 'advanced_education_pct']

Sample data:
           Country Name Country Code  Year  advanced_education_pct
0                 Aruba          ABW  1960                     NaN
1           Afghanistan          AFG  1960                     NaN
2                Angola          AGO  1960                     NaN
3               Albania          ALB  1960                     NaN
4               Andorra          AND  1960                     NaN
5  United Arab Emirates          ARE  1960                     NaN
6             Argentina          ARG  1960                     NaN
7               Armenia          ARM  1960                     NaN
8        American Samoa          ASM  1960                     NaN
9   Antigua and Barbuda          ATG  1960                     NaN

Data types:
Country Name               object
Country Code               object
Year                        int64
advanced

## Step 3: Merge the Two Education Datasets

Combine both education datasets on Country Code and Year.

In [4]:
# Merge the two education datasets
merged_df = advanced_long.merge(
    basic_long[['Country Code', 'Year', 'basic_education_pct']],
    on=['Country Code', 'Year'],
    how='outer'  # Use outer join to keep all data from both datasets
)

print(f"Merged dataset shape: {merged_df.shape}")
print(f"\nColumns: {merged_df.columns.tolist()}")
print(f"\nFirst 10 rows:")
print(merged_df.head(10))
print(f"\nData summary:")
print(merged_df.describe())

Merged dataset shape: (13975, 5)

Columns: ['Country Name', 'Country Code', 'Year', 'advanced_education_pct', 'basic_education_pct']

First 10 rows:
  Country Name Country Code  Year  advanced_education_pct  basic_education_pct
0        Aruba          ABW  1960                     NaN                  NaN
1        Aruba          ABW  1961                     NaN                  NaN
2        Aruba          ABW  1962                     NaN                  NaN
3        Aruba          ABW  1963                     NaN                  NaN
4        Aruba          ABW  1964                     NaN                  NaN
5        Aruba          ABW  1965                     NaN                  NaN
6        Aruba          ABW  1966                     NaN                  NaN
7        Aruba          ABW  1967                     NaN                  NaN
8        Aruba          ABW  1968                     NaN                  NaN
9        Aruba          ABW  1969                     NaN    

## Step 4: Remove 2024 and Analyze Missing Data

Filter out 2024 data and review missing data statistics.

In [5]:
# Remove year 2024 (incomplete/projected data)
merged_df = merged_df[merged_df['Year'] != 2024].copy()

print(f"After removing 2024:")
print(f"Dataset shape: {merged_df.shape}")
print(f"Year range: {merged_df['Year'].min()} - {merged_df['Year'].max()}")

# Analyze missing data
print(f"\n{'='*60}")
print("Missing data summary:")
print(merged_df.isnull().sum())
print(f"\nMissing data percentages:")
print((merged_df.isnull().sum() / len(merged_df) * 100).round(2))

# Calculate data coverage per country for informational purposes
country_coverage = merged_df.groupby('Country Code').apply(
    lambda x: pd.Series({
        'country_name': x['Country Name'].iloc[0],
        'total_rows': len(x),
        'advanced_ed_coverage': x['advanced_education_pct'].notna().sum() / len(x) * 100,
        'basic_ed_coverage': x['basic_education_pct'].notna().sum() / len(x) * 100,
        'overall_coverage': x.notna().all(axis=1).sum() / len(x) * 100
    })
)

print(f"\nCountry coverage statistics:")
print(country_coverage.describe())

# Keep all countries - no filtering based on missing data
cleaned_df = merged_df.copy()

print(f"\n{'='*60}")
print(f"Total countries: {cleaned_df['Country Code'].nunique()}")
print(f"Dataset shape: {cleaned_df.shape}")
print(f"{'='*60}")

After removing 2024:
Dataset shape: (13760, 5)
Year range: 1960 - 2023

Missing data summary:
Country Name                  0
Country Code                  0
Year                          0
advanced_education_pct    11483
basic_education_pct       11468
dtype: int64

Missing data percentages:
Country Name               0.00
Country Code               0.00
Year                       0.00
advanced_education_pct    83.45
basic_education_pct       83.34
dtype: float64

Country coverage statistics:
       total_rows  advanced_ed_coverage  basic_ed_coverage  overall_coverage
count       215.0            215.000000         215.000000        215.000000
mean         64.0             16.547965          16.656977         16.475291
std           0.0             16.400158          16.506327         16.440923
min          64.0              0.000000           0.000000          0.000000
25%          64.0              3.125000           3.125000          3.125000
50%          64.0             10.937500

## Step 5: Standardize Column Names and Final Validation

Clean up column names and perform final data quality checks.

In [6]:
# Standardize column names to snake_case
cleaned_df.columns = ['country_name', 'country_code', 'year', 
                       'advanced_education_pct', 'basic_education_pct']

# Sort by country and year for better organization
cleaned_df = cleaned_df.sort_values(['country_code', 'year']).reset_index(drop=True)

# Final data validation
print("=" * 60)
print("FINAL CLEANED DATASET")
print("=" * 60)
print(f"\nShape: {cleaned_df.shape}")
print(f"Countries: {cleaned_df['country_code'].nunique()}")
print(f"Year range: {cleaned_df['year'].min()} - {cleaned_df['year'].max()}")
print(f"\nColumn names:")
print(cleaned_df.columns.tolist())
print(f"\nData types:")
print(cleaned_df.dtypes)
print(f"\nFirst 15 rows:")
print(cleaned_df.head(15))
print(f"\nSummary statistics:")
print(cleaned_df.describe())
print(f"\nMissing values:")
missing_summary = pd.DataFrame({
    'Column': cleaned_df.columns,
    'Missing Count': cleaned_df.isnull().sum().values,
    'Missing %': (cleaned_df.isnull().sum() / len(cleaned_df) * 100).round(2).values
})
print(missing_summary)
print(f"\nSample of countries included:")
print(cleaned_df[['country_name', 'country_code']].drop_duplicates().head(20))

FINAL CLEANED DATASET

Shape: (13760, 5)
Countries: 215
Year range: 1960 - 2023

Column names:
['country_name', 'country_code', 'year', 'advanced_education_pct', 'basic_education_pct']

Data types:
country_name               object
country_code               object
year                        int64
advanced_education_pct    float64
basic_education_pct       float64
dtype: object

First 15 rows:
   country_name country_code  year  advanced_education_pct  \
0         Aruba          ABW  1960                     NaN   
1         Aruba          ABW  1961                     NaN   
2         Aruba          ABW  1962                     NaN   
3         Aruba          ABW  1963                     NaN   
4         Aruba          ABW  1964                     NaN   
5         Aruba          ABW  1965                     NaN   
6         Aruba          ABW  1966                     NaN   
7         Aruba          ABW  1967                     NaN   
8         Aruba          ABW  1968          

## Step 6: Export Cleaned Dataset

Save the final cleaned dataset.

In [ ]:
# Export to CSV
output_filename = '../data/clean/women_education.csv'
cleaned_df.to_csv(output_filename, index=False)

print(f"✓ Dataset successfully exported to: {output_filename}")
print(f"\nFinal dataset info:")
print(f"  - Rows: {len(cleaned_df):,}")
print(f"  - Countries: {cleaned_df['country_code'].nunique()}")
print(f"  - Years: {cleaned_df['year'].min()} to {cleaned_df['year'].max()}")
print(f"  - Features: {', '.join(cleaned_df.columns.tolist())}")
print(f"\nThe dataset is ready!")